# Modules and Calling External Programs
## Topics
- Modules - indepedent collections of related functions
- Using Modules - math & collections
- Making your own module
- Interacting with other software using __subprocess__
- Making your own BLAST module


## Introduction

We're covering a lot of ground today!

First up we'll be discussing modules and how they are used to organize code.

Then we're getting into ways to call external programs from within Python. We'll also cover saving the output of these programs, either to new files or within python's memory.

## Modules

In all of the examples so far, we defined our functions right above the code that we hoped to execute. If you have many functions, you can see how this would get messy in a hurry. 
Furthermore, part of the benefit of functions is that you can call them multiple times within a program to execute the same operations without writing them all out again. 

But wouldn't it be nice to share functions across programs, too? 

For example, working with genomic data means lots of time getting sequence out of FASTA files, and shuttling that sequence from program to program. Many of the programs we work with overlap to a significant degree, as they need to parse FASTA files, calculate evolutionary rates, and interface with our lab servers, for example -- all of which means that many of them share functions. And if the same function exists in two or more different programs, we hit the same problems that we hit before: complex debugging, decreased readability, and, of course, too much typing.

Modules solve these problems. In short, they're collections of functions and variables (and often objects, which we'll get to towards the end of the course) that are kept together in a single file that can be read and __import__ed by any number of programs.

### The Basics: Using the math module

To illustrate the basics, we'll go through the use of the __math__ module, a module which we use almost all the time. To use a function or variable in the __math__ module, use the syntax 
__math.__*NameOfThing

In [1]:
##to use a module in your code, first import it
import math

x = 5

##Modules usually contain functions
log10 = math.log10(x)
cos = math.cos(x)

##sometimes modules contain variables
pi = math.pi
e = math.e

print log10, cos, pi, e

0.698970004336 0.283662185463 3.14159265359 2.71828182846


In [2]:
math.cos?

In [5]:
dir()

['In',
 'Out',
 '_',
 '_3',
 '_4',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__name__',
 '__package__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 '_sh',
 'cos',
 'e',
 'exit',
 'get_ipython',
 'log10',
 'math',
 'pi',
 'quit',
 'x']

In [6]:
def foo(x):
    y = 7
    
foo(9)
print y

NameError: name 'y' is not defined

### The collections module

Another useful module is the __collections__ module. It has a bunch of new data types that are, as you might guess from the name, collections of other things. We will cover two of the most commonly used objects: __Counter__ and __defaultdict__. Let's start with __Counter__, which counts things.

In [7]:
import collections
 
my_genera = ['Helicobacter', 'Escherichia', 'Lactobacillus',
             'Lactobacillus', 'Oryza', 'Wolbachia', 'Oryza',
             'Rattus', 'Lactobacillus', 'Drosophila']
 
c = collections.Counter(my_genera)
print c
##Note that placing the list into Counter() immediately gets
##you the count.

Counter({'Lactobacillus': 3, 'Oryza': 2, 'Drosophila': 1, 'Escherichia': 1, 'Rattus': 1, 'Wolbachia': 1, 'Helicobacter': 1})


The collections module gives us a new data type, __Counter__, that counts things. It is essentially a dictionary where the key is some element we are recording and the value is the count of how often it appears. Remember that list of amino acids we got the count for in the exercises in Section 2.1? There, we created a dictionary where every key was initialized with a value of zero, and then proceeded to add one for each observance. Here, we can just use a __Counter__ to get the count of each unique element in the list.

In [8]:
##This is how we did a count in a dictionary. Many more lines of code!
counts = {}
 
for genus in my_genera:
    if genus not in counts:
        counts[genus] = 0
    counts[genus] += 1

print "The dictionary", counts

The dictionary {'Lactobacillus': 3, 'Oryza': 2, 'Drosophila': 1, 'Escherichia': 1, 'Rattus': 1, 'Wolbachia': 1, 'Helicobacter': 1}


Using a __Counter__ is faster to write and saves us writing this bit of code every time we want to something. Another big advantage of the __Counter__ type is that it makes it really easy to sort by frequency:

In [10]:
my_seq = ['MET', 'GLU', 'VAL', 'LYS', 'ARG', 'GLU', 'HIS', 'TRP', 'ALA',
          'THR', 'ARG', 'LEU', 'GLY', 'LEU', 'ILE', 'LEU', 'ALA', 'MET',
          'ALA', 'GLY', 'ASN', 'ALA', 'VAL', 'GLY', 'LEU', 'GLY', 'ASN',
          'PHE', 'LEU', 'ARG', 'PHE', 'PRO', 'VAL', 'GLN', 'ALA', 'ALA',
          'GLU', 'ASN', 'GLY', 'GLY', 'GLY', 'ALA', 'PHE', 'MET', 'ILE',
          'PRO', 'TYR', 'ILE', 'ILE', 'ALA', 'PHE', 'LEU', 'LEU', 'VAL',
          'GLY', 'ILE', 'PRO', 'LEU', 'MET', 'TRP', 'ILE', 'GLU', 'TRP',
          'ALA', 'MET', 'GLY', 'ARG', 'TYR', 'GLY', 'GLY', 'ALA', 'GLN',
          'GLY', 'HIS', 'GLY', 'THR', 'THR', 'PRO', 'ILE', 'VAL', 'PHE',
          'LEU', 'ILE', 'THR', 'MET', 'PHE', 'ILE', 'ASN', 'VAL', 'SER',
          'ILE', 'LEU', 'ILE', 'ARG', 'GLY', 'ILE', 'SER', 'LYS', 'GLY',
          'ILE', 'GLU', 'ARG', 'PHE', 'ALA', 'LYS', 'ILE', 'ALA', 'MET',
          'PRO', 'THR', 'LEU', 'PHE', 'ILE', 'LEU', 'ALA', 'VAL', 'PHE',
          'LEU', 'VAL', 'ILE', 'ARG', 'VAL', 'PHE', 'LEU', 'LEU', 'GLU',
          'THR', 'PRO', 'ASN', 'GLY', 'THR', 'ALA', 'ALA', 'ASP']

c = collections.Counter(my_seq)
 
print c
print
print
print c.most_common()

for amino, count in c.most_common():
    print amino, count

Counter({'GLY': 16, 'ILE': 16, 'ALA': 15, 'LEU': 15, 'PHE': 10, 'VAL': 9, 'THR': 7, 'MET': 7, 'ARG': 7, 'PRO': 6, 'GLU': 6, 'ASN': 5, 'LYS': 3, 'TRP': 3, 'GLN': 2, 'HIS': 2, 'TYR': 2, 'SER': 2, 'ASP': 1})


[('GLY', 16), ('ILE', 16), ('ALA', 15), ('LEU', 15), ('PHE', 10), ('VAL', 9), ('THR', 7), ('MET', 7), ('ARG', 7), ('PRO', 6), ('GLU', 6), ('ASN', 5), ('LYS', 3), ('TRP', 3), ('GLN', 2), ('HIS', 2), ('TYR', 2), ('SER', 2), ('ASP', 1)]
GLY 16
ILE 16
ALA 15
LEU 15
PHE 10
VAL 9
THR 7
MET 7
ARG 7
PRO 6
GLU 6
ASN 5
LYS 3
TRP 3
GLN 2
HIS 2
TYR 2
SER 2
ASP 1


*Counter.*__most_common()__ returns a list of tuples, sorted in order by highest count to lowest count.

The other __collections__ type we will cover is __defaultdict__, which is also like a dictionary, but has a default type for a key that we haven't seen before (with a normal dictionary, if you try to read something where the key isn't in the dict, then you get an error). Let's think about how we'd make a dictionary where each key is a genus, and the value is a list of species in that genus:

In [11]:
import collections
 
my_species = [('Helicobacter','pylori'), ('Escherichia','coli'),
              ('Lactobacillus', 'helveticus'),
              ('Lactobacillus', 'acidophilus'),
              ('Oryza', 'sativa'), ('Wolbachia', 'pipientis'),
              ('Oryza', 'glabberima'), ('Rattus', 'norvegicus'),
              ('Lactobacillus','casei'), ('Drosophila','melanogaster')]
 
# Below, we put the list into a normal dictionary, 
# with genera as keys and species as values
d1 = {}
for genus, species in my_species:
    if genus not in d1:
        d1[genus] = []
    d1[genus].append(species)

print "normal dictionary -- ", d1

normal dictionary --  {'Lactobacillus': ['helveticus', 'acidophilus', 'casei'], 'Oryza': ['sativa', 'glabberima'], 'Drosophila': ['melanogaster'], 'Escherichia': ['coli'], 'Rattus': ['norvegicus'], 'Wolbachia': ['pipientis'], 'Helicobacter': ['pylori']}


With a __defaultdict__, we can once again save the line in the for loop where we check for a non-existent key:

In [12]:
d2 = collections.defaultdict(list)
 
for genus, species in my_species:
    d2[genus].append(species)

print "default dict -- ", d2

default dict --  defaultdict(<type 'list'>, {'Lactobacillus': ['helveticus', 'acidophilus', 'casei'], 'Oryza': ['sativa', 'glabberima'], 'Drosophila': ['melanogaster'], 'Escherichia': ['coli'], 'Rattus': ['norvegicus'], 'Wolbachia': ['pipientis'], 'Helicobacter': ['pylori']})


Moreover, if we check for the species in a genus that has no species, we no longer receive an error.

In [13]:
print d2['Saccharomyces']

[]


One thing to look at is the line where we actually declare the defaultdict: here we've given it another type, and if we use a key that's not in the dictionary already, it will initialize it to be an empty variable of that type. Most often, this will be a list, but you could imagine uses for other types, like a string, an integer (here "empty" actually would mean 0), or even another dict. It's possible to even have a defaultdict of defaultdicts!

## Making Your Own Modules

Now that you know the basics of __import__ing and using a module, you will learn how to write our own modules, which is almost just as easy!

Any file of python code with a *.py* extension can be __import__ed as a module from your script. When you invoke an __import__ operation from a program, all the statements in the __import__ed module are executed immediately. The program also gains access to names assigned in the module (names can be functions, variables, classes, etc.), which can be invoked in the program using the syntax *module.name*. Find the following script in the file *greeting_module.py*:

```python
print 'The top of the greeting_module has been read.'
 
def hello(name):
 greeting = "Hello {}!".format(name)
 return greeting
 
def ahoy(name):
 greeting = "Ahoy-hoy {}!".format(name)
 return greeting
 
x = 5
 
print 'The bottom of the greeting_module has been read.'
```

The following script will call the module **greeting_module** and use the functions and variables located within the module.

In [1]:
# Make sure you reset your kernel before running this cell,
# or you won't see the output from greeting_module.
import greeting_module

greeting = greeting_module.hello('Christopher')
print greeting
print

x = 1
print 'x within greeting_module:', greeting_module.x
print 'x within the __main__ module:', x

The top of the greeting_module has been read.
The bottom of the greeting_module has been read.
Hello Christopher!

x within greeting_module: 5
x within the __main__ module: 1


Notice that it runs through all of greeting module first, so anything that is printed out in *greeting_module.py* is also printed out before anything in test.py is run.

And that's it! See-- no more messy function declarations at the beginning of your script. Now if you need any other program to say "hi" to you, all you need to do is __import__ the greeting module.

---
## Informative Interlude: The Kernel

A kernel is a program that translates instructions from a user (or storage media) into machine operations, and produces output back to the user (or other program). The IPython Kernel that we are using for this notebook reads Python code that is given to it and translates it into instructions for the computer, then prints any output text. The vanilla Python Kernel does the same thing, but doesn't have fancy features like tab-completion and multi-line input.

Most of the time, when you run a Python program, the Python Kernel reads a script (the _.py_ file) and exits when reaching an Exception, an exit instruction, or the end of the file. When you run python on the terminal by typing "python" or "ipython", instead of reading a script, the kernel reads from your direct input (called stdin). After reading a line you type in, this "interactive kernel" (actually an interpreter) pauses and waits for another input, rather than simply exiting, even if the last input caused an exception.

In these Jupyter notebooks, we are using the IPython Kernel in a third way. It is as if we have started a Python interpreter by typing "ipython" in a terminal, and it is just sitting in the background waiting. When we run a cell of Python code, it is as if we copy that cell and paste it into our idle interpreter.

This is why you need to restart your kernel before running cells with __import__ statements to make sure those imports worked correctly. Python is lazy and if it sees you are trying to __import__ a module that has already been __import__ed it will skip it. This may have also been the cause of some of your confounding errors throughout this class. If you ran cells out of order, _x_ may not have been what you thought it was. Or maybe you got stuck somewhere, unable to run any code, because some other cell was caught in an infinite loop, or waiting for some __raw_input()__.

Here's where you can [read up on how the IPython Kernel actually works](https://ipython.org/ipython-doc/3/development/how_ipython_works.html "How IPython works") in Jupyter Notebooks.

---

## Using Modules Beyond 'import'

Here, we will cover how to modify your __import__ statement to make using modules less clunky in your code.

First, what if you only want one function from a given module? Let's say, as an Alexander Graham Bell loyalist, you really only dealt in 'ahoys' rather than 'hellos.' We need to use a modified syntax for retrieving _only_ the __ahoy__ function from the module, without cluttering things up by loading the newfangled __hello__ function preferred by T.A. Edison's entourage.

In [1]:
# Again, make sure to restart your kernel before running this.
from greeting_module import ahoy

greeting = ahoy('everybody')
# if you grab a function from a module with a 'from' statement,
# you don't need to use the <module>.<function> syntax
print greeting
print
print hello('Christopher')

The top of the greeting_module has been read.
The bottom of the greeting_module has been read.
Ahoy everybody!



NameError: name 'hello' is not defined

The __from__ _module_ __import__ _object_ syntax has two differences from just plain __import__ _module_. First we see that we can now write __ahoy('everybody')__ directly, instead of having to write __greeting_module.ahoy('everybody')__. Second, we don't have access to __hello__ or any of the other objects in the greeting module. If we wanted to access both functions this way, we could __import__ them both in one statement like so:

In [1]:
# Restart your kernel
from greeting_module import ahoy, hello
print hello("You")
print ahoy("Matey")

The top of the greeting_module has been read.
The bottom of the greeting_module has been read.
Hello You!
Ahoy Matey!


Or, what if there were a lot of functions from the __greeting_module__ we wanted to use, but didn't want to write out the full name? Rather than writing out all of the function names to import individually (there could be a lot of them), we can use the asterisk wildcard (_*_) to refer to them.

In [2]:
# Restart your kernel
x = 1
from greeting_module import *

print x

print ahoy('everybody')
print hello('everybody')

5
Ahoy everybody!
Hello everybody!


While this may be useful if we are familiar with the contents of the module, including all of the names inside, there are a few reasons to be careful about using the __from__ _module_ __import *__ syntax. 

First, if the module contains a lot of variables that we don't need to use, we will needlessly allocate memory to storing the information.

Second, and perhaps more importantly, if the module being imported contains variables with the same names as those inside your program, you will lose access to the original values of those variables. This is what happens to _x_, above.

### Changing Names When Importing

It is possible to avoid name conflicts by changing the name of objects you import using the __as__ keyword.

In [3]:
# Restart your kernel
from greeting_module import hello as hi

def hello(name):
    greeting = "What's up, Doc?"
    return greeting

print hi('World')
print hello('World')

Hello World!
What's up, Doc?


This can also be useful to shorten commonly used module names when you want many things in the module, but don't want to run into conflicts by using __from__ _module_ __import *__.

In [5]:
import greeting_module as gm

print gm.x

5


In [ ]:
import greeting_module

## Where to Store Your Modules: using PYTHONPATH

Over time, you'll end up accumulating lots of these modules, and they'll tend to fall together in meaningful collections.

For example, 
- You might have a module for all your functions related to reading and parsing files, called *files_tools.py*. 
- You might have another for common sequence-related tasks, called *sequence_tools.py*. 

Python keeps its modules installed in a system directory.  If you are running Python on a remote server, you may not even have permissions to add files to this directory. Therefore, it's useful and simpler to just create your own Python modules directory and then let your operating system environment know about it. 

### Adding a folder to your Python Path: Linux and OSX
Python doesn't look through every possible file and folder on your computer to find saved modules. Rather, it looks at a select set of folders, which are stored in your system's **PYTHONPATH** variable. You can add a new folder for Python to look in by adding a few lines to your *.bashrc* file in your home directory with the following terminal commands:
```bash
sudo nano ~/.bashrc
```
Mac Users use
```bash
sudo nano ~/.bash_profile
```

This will open your *.bashrc* file, which contains a set of bash commands run whenever you start a shell (terminal) to setup your environment. Add the following lines to the end of the file, substituting the path to your module. 

```bash
PYTHONPATH=$PYTHONPATH:/path/to/your/module

export PYTHONPATH
```

The PYTHONPATH variable is the list of all folders that Python knows contain modules. The above sets the PYTHONPATH variable (variables are indicated by a "$" in bash) to include the folder your module is in.

Hit CONTROL + X to quit, then type 'y' and hit enter to save.

Next, reload your *.bashrc*

```bash
source ~/.bashrc
```

And check if it worked using **echo**

```bash
echo $PYTHONPATH
```

And with that, any file that ends up in this directory will be treated as a module by Python. And though this is a good final resting place for your polished modules, you can also prototype them by simply saving them in your current working directory, and moving them over when you're happy with them.

### Adding a folder to your Python Path: Windows

On Windows, you can do this using the command prompt, but it's safer to use the built in graphical interface.

To get there:
- Open up "My Computer" or "This PC"
- Right-click in the background and select "Properties"
- On the "Advanced" tab, click the "Environment Variables" button
- Add or edit your PYTHONPATH variable, seperating directories with a semicolon ";" as seen below

![Environment Variables](WindowsEnvVars.png)

# System Calls With the subprocess Module

In this section we will cover how to use the **subprocess** module to interact with other software, making system calls, collecting data, and automating your entire analysis pipeline.

In bioinformatics we will often want to use other peoples software, whether that's Python packages like Biopython or packages written in other languages, like the highly efficient C. Rather than reinvent the wheel, we use many specialized packages to quickly and easily perform tasks such as alignment, SNP calling, or phylogenetics. Even simple Unix tools like **wc** can be useful to count the number of genes in a GFF file or length of a FASTA sequence.

Most bioinformatics tools, like Unix tools, are run through the command line. Given that you might want to repeat an analysis using some of these tools on, for example, hundreds of genes, manually entering the commands for your entire pipeline is not only boring and error prone but a waste of your time.

Here we will teach you to automate your pipeline by using Python to run these command line tools for you.

---
## Advantages to Automation

**Reproducability**

Perhaps the greatest advantage to a scientist in automating analysis is that the analysis can be reproduced exactly. Your exact methods are laid out in your Python script, where you and others can scrutinize, repeat, and modify them.

**Less Tedium**

From printing out the name of every gene expressed over a certain level, to BLASTING those genes against the NCBI database, to sorting and counting the resulting hits, scripting saves you a huge amount of tedious labor. Nobody wants to type, or even copy and paste, hundreds of BLAST queries.

**Consistency**

As well as being incredibly mind-numbing, manually running bioinformatics tools is dangerous. What if you accidentally type 'Gasterosteus_aculeatus_CA_SNPs.vcf' instead of 'Gasterosteus_aculeatus_AC_SNPs.vcf', accidentally substituting your California population for your Atlantic Coast population? Or 'clean_reads.py expensive_dataset.fq > expensive_dataset.fq' instead of 'clean_reads.py expensive_dataset.fq > expensive_dataset.clean.fq'? There are *thousands* of ways you can accidentally screw up your analysis to either ruin your day or produce erroneous results.

Automation reduces the risk of stupid typos and other accidents. You won't forget to include mydata.part.14.bam in the analysis when you run **results = [analyse(data) for data in mydata]**.

**Parallelization**

Modern computers, even budget laptops, now have multiple processors, which means you can run several or even hundreds of analyses at once (if you have access to a supercomputing cluster)! Python provides a number of tools to help you manage these processes and make the most out of parallel computing.

---

## subprocess.check_output()
The easiest way to make system calls is with the __check_output()__ function in the __subprocess__ module

In [1]:
import subprocess as sp
 
output = sp.check_output('ls', shell=True)

print output

4.2_Solutions.ipynb
greeting_module.pyc
Lecture 4.2 Modules and Calling External Programs.ipynb
my_modules
Optional_Advanced_System_Calls.ipynb
test.fasta
test_output.py
WindowsEnvVars.png
yeast.aa
yeast_genome.nsq
yeast.nt



Here, we used __subprocess.check_output()__ to run the command **ls**, and captured the output in the *output* variable.

You'll notice that **check_output()** takes one mandatory argument: the command you want to run as you would type it into your terminal shell. The __shell__ keyword argument we will leave as __True__ for this tutorial. Without it, the process is created directly by the operating system, and any symbols or commands that the shell would recognize (e.g. spaces, ">", and "|") result in an error. Because spaces are a symbol recognized by the shell, when calling __call()__ without __shell=True__, the first argument should be a list of command line arguments that would be separated by spaces spaces on the terminal. So **ls -a -1 /home/james** would become **['ls', '-a', '-1', '/home/james']**.

### Redirecting Output to Files
Since we are using __shell=True__, you can redirect the output of a command to file exactly as you would on the terminal.

To demonstrate this, we will be asking Python to do something very meta: run another Python script! Here's the script we will be running, which should be saved in your 5.2 directory as 'test_output.py'.

In [2]:
%%writefile test_output.py
#!/usr/bin/env python
print 'this is a test'

Overwriting test_output.py


Remember that we can redirect the output of a command to a file with __>__.

In [3]:
import subprocess as sp

command = 'python test_output.py > out.txt'
output = sp.check_output(command, shell=True)
print output

This time there is no output to print, since we redirected it. Check 'out.txt' to make sure the output went where you expected it.

### Chaining Commands With Pipes
We often want to do something else with the output of a program, either parsing it and reformating it, performing a second step in the analysis, or turning it into a figure. To do this, we will *pipe* the output of one command directly into another command.

A *pipe* is used to send the output of one program into the input of another. We learned in the first lecture that this is done on the Unix command line with the **|** character. For example, this script should count the number of files that contain the word "yeast" in the current directory.

In [5]:
import subprocess as sp

command = "ls -1 | grep 'yeast' | wc -l"
print sp.check_output(command, shell=True)

3



If these Unix commands are new to you, try breaking this chained command down to figure out what each step is doing.

## BLAST
Good ol'e Basic Local Alignment Search Tool, now that's bioinformatics even Grandma can get down with. In its most basic form, BLAST takes a short sequence of either amino acids or nucleotides, searches a database of reference sequences, and returns the most likely matches for the query sequence. It's brilliant, and the only trouble is that we're often interested in BLASTing hella query sequences, and much less interested in the mundanity of parsing the output that comes back to us. Fortunately, we can easily script the execution of BLAST such that we can run queries many times with different sequences and settings, and also parse the output back into dictionaries to manipulate and use for downstream analysis.

The first part of this section will show us how to install a new program in our UNIX environment, and the second part will let us practice using **subprcess** to script the execution of BLAST.

### Installing BLAST
I suspect most everyone here is accustomed to using the NCBI BLAST web interface, but it's just another program, whether it's running on an NCBI web server, or on our laptops. First we will install BLAST locally, such that we have ready access to it from anywhere on our computers.

#### Windows

Windows users have the easiest time installing BLAST.

Go to the PythonCourse/ProgramFiles/windows folder. Run the ncbi-blast-2.4.0+-win64.exe file (either by double clicking it in a file browser or invoking it within a shell), and follow the install GUI. It should automatically install all the different BLAST programs, and add them to your path. 

To verify your install worked, open a fresh terminal a type 'makeblastdb.' The program should give you an error (as you gave it no input), but should display the command line help. We will cover how to use the different BLAST programs below.

#### Mac and Linux

In the PythonCourse/ProgramFiles directory, there is a *.tar.gz* file containing the BLAST executable for your platform. The *.tar.gz* file extension tells us that the file has been archived (the *.tar* part) and compressed (the *.gz* part). The UNIX command to uncompress and expand the file is:

**tar -zxvf [filename]**

And we can tell **tar** where to dump the output with the **-C** option, like so:

**tar -zxvf [filename] -C [output directory]**

Let's use __check_output()__ to do this now. 

In [6]:
import subprocess as sp

# Make sure this is right for you.
blast_archive = ('~/Dropbox/2018_winter_python/ProgramFiles/' + \
                 'linux/ncbi-blast-2.2.26+-x64-linux.tar.gz')
blast_destination = '~/Dropbox/2018_winter_python/ProgramFiles'

command = 'tar -zxvf {} -C {}'.format(blast_archive, blast_destination)

# This may take a few seconds
print sp.check_output(command, shell=True)

ncbi-blast-2.2.26+/
ncbi-blast-2.2.26+/bin/
ncbi-blast-2.2.26+/bin/blastn
ncbi-blast-2.2.26+/bin/rpstblastn
ncbi-blast-2.2.26+/bin/convert2blastmask
ncbi-blast-2.2.26+/bin/blastdbcmd
ncbi-blast-2.2.26+/bin/tblastn
ncbi-blast-2.2.26+/bin/makeblastdb
ncbi-blast-2.2.26+/bin/makeprofiledb
ncbi-blast-2.2.26+/bin/segmasker
ncbi-blast-2.2.26+/bin/windowmasker
ncbi-blast-2.2.26+/bin/update_blastdb.pl
ncbi-blast-2.2.26+/bin/blastdb_aliastool
ncbi-blast-2.2.26+/bin/rpsblast
ncbi-blast-2.2.26+/bin/psiblast
ncbi-blast-2.2.26+/bin/dustmasker
ncbi-blast-2.2.26+/bin/makembindex
ncbi-blast-2.2.26+/bin/blastp
ncbi-blast-2.2.26+/bin/tblastx
ncbi-blast-2.2.26+/bin/blastx
ncbi-blast-2.2.26+/bin/blastdbcheck
ncbi-blast-2.2.26+/bin/deltablast
ncbi-blast-2.2.26+/bin/blast_formatter
ncbi-blast-2.2.26+/bin/legacy_blast.pl
ncbi-blast-2.2.26+/doc/
ncbi-blast-2.2.26+/doc/README.txt
ncbi-blast-2.2.26+/ChangeLog
ncbi-blast-2.2.26+/ncbi_package_info
ncbi-blast-2.2.26+/README
ncbi-blast-2.2.26+/LICENSE



Within the BLAST directory we've got a *ChangeLog* file that will tell us what version of BLAST we've got, a *doc* directory with documentation, a *data* directory with the various matrices to use for esimating substituion rates, and then the *bin* directory. *bin* stands for binary, and directories named *bin* are conventionally used to store executable binary files. Try typing **which less** on a terminal and you will see that **less**, just like all your terminal commands (**ls**, **mv**, **cat**, etc), is actually an executable binary, probably in a directory like */usr/bin*.

Just like we added our modules to our PYTHONPATH environment variable, we can add executables to our shell's PATH. To do this, in a terminal, type:

```bash
sudo nano ~/.bashrc
```

Then add the following lines. __MAKE CERTAIN YOU TYPE THIS EXACTLY CORRECTLY! BREAKING YOUR PATH CAN BE HARD TO FIX__:
```bash
PATH=$PATH:~/Dropbox/2018_winter_python/ProgramFiles/ncbi-blast-2.2.26+/bin
export PATH
```

Hit CONTROL + x to quit, then type 'y' and hit ENTER to save.

Reload your *.bashrc*:

```bash
source ~/.bashrc
```

And check if it worked using **which**:

```bash
which makeblastdb
which blastn
```


### Creating a BLAST Database

Now that we have these programs available, let's start using them.

Since BLAST is going to compare a query sequence to a database of sequences, collected from say, a genome, or a group of genomes, we first need to create the reference database to BLAST against. Many of these are available for download, but we're going to create our own from the *S. cerevisiae* genome.

The yeast genome is available on the NCBI FTP server, though we have included it in your course folder. *yeastn.nt* is the fasta formated genome, while *yeast.aa* is the fasta formated collection of all proteins.

To make your own local BLAST database, use the **makeblastdb** program we just installed.

In [1]:
import subprocess as sp

command = 'makeblastdb -in yeast.nt -out yeast_genome -dbtype nucl'

print sp.check_output(command,shell=True)



Building a new DB, current time: 01/11/2018 15:04:18
New DB name:   yeast_genome
New DB title:  yeast.nt
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1073741824B
Adding sequences from FASTA; added 17 sequences in 0.186441 seconds.



Now we have a fully operational BLAST installation and a yeast nucleotide database to search against. We can see that BLAST has created some additional files for our yeast database in our working directory (*.nhr*, *.nin*, and *.nsq*).

### Running BLAST!

Now let's query the database with __blastn__. We will have to make an input file to serve as our query.

In [2]:
import subprocess as sp

testseq = 'TGTTGTATTACGGGCTCGAGTAATACCGGAGTGTCTTGACAATCCTAATATAAA' \
          'CGGTCTTAGGGAAGTAACCAGTTGTCAAAACAGTTTATCAGATTAATTCACGGA' \
          'ATGTTACTTATCTTATATATTATATAAAATATGAtATCATATTAAGTGGTGGAA' \
          'GCGCGGAATCTCGGATCTAAACTAATTGTTCAGGCATTTATACTTTTGGTAGTT' \
          'CAGCTAGGGAAGGACGGGTTTTATCTCATGTTGTTCGTTTTGTTATAAGGTTGT' \
          'TTCATATGTGTTTTATGAACGTTTAGGATGACGTATTGTCATACTGACATATCT' \
          'CATTTTGAGATACAACA'

def blastn(querySeq):
    fh = open('query.seq', 'w')
    fh.write(testseq)
    fh.close()
    
    command = 'blastn -db yeast_genome -query query.seq'
    out = sp.check_output(command, shell=True)
    return out

print blastn(testseq)

BLASTN 2.2.26+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: yeast.nt
           17 sequences; 12,155,026 total letters



Query= 
Length=341
                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

  gi|6321173|ref|NC_001139.1| Saccharomyces cerevisiae chromosome...   623    8e-179
  gi|7276232|ref|NC_001137.2| Saccharomyces cerevisiae chromosome...   597    5e-171
  gi|7839148|ref|NC_001136.2| Saccharomyces cerevisiae chromosome...   592    2e-169
  gi|6322623|ref|NC_001143.1| Saccharomyces cerevisiae chromosome...   584    4e-167
  gi|6322016|ref|NC_001141.1| Saccharomyces cerevisiae chromosome...   584    4e-167
  gi|6319247|ref|NC_001133.1| Saccharomyces cerevisiae chromosome...   584    4e-167
  gi|6324971|ref|NC_001148.1| Saccharomyces cerevis

You can imagine how you use this function in a loop to query as list of sequences, or modify the function to vary the reference database, or even vary some of the BLAST settings (for example, the minimum E-value to report).

We have caught the output in Python, which might be helpful when we want to systematically sort through or analyze the results of each BLAST hit. In other circumstances, however, we might want to write the results directly to an output file. Let's try that:

In [3]:
import subprocess as sp

testseq = 'TGTTGTATTACGGGCTCGAGTAATACCGGAGTGTCTTGACAATCCTAATATAAA' \
          'CGGTCTTAGGGAAGTAACCAGTTGTCAAAACAGTTTATCAGATTAATTCACGGA' \
          'ATGTTACTTATCTTATATATTATATAAAATATGAtATCATATTAAGTGGTGGAA' \
          'GCGCGGAATCTCGGATCTAAACTAATTGTTCAGGCATTTATACTTTTGGTAGTT' \
          'CAGCTAGGGAAGGACGGGTTTTATCTCATGTTGTTCGTTTTGTTATAAGGTTGT' \
          'TTCATATGTGTTTTATGAACGTTTAGGATGACGTATTGTCATACTGACATATCT' \
          'CATTTTGAGATACAACA'

def blastn(querySeq, outfile):
    fh = open('query.seq', 'w')
    fh.write(testseq)
    fh.close()
    
    command = 'blastn -db yeast_genome -query query.seq > ' + outfile
    sp.check_output(command, shell=True)

blastn(testseq, 'blastn_output.txt')

We simply included a second argument to our function, the name of the output file, and used the redirection symbol (__>__) in our shell command to send the output to the file. Note that this function doesn't return anything.

---

## Exercises

__1) Math Overload__

Use the **math** module to compute the following:

* $4^8$
* $log_{10}(3)$
* $e^3$
* $cos(\pi)$
* $ln(e^3)$

__2. Making Directories__

Bioinformatics projects require an organized system of files and directories. Work using the **subprocess** module to create a mock file system. Use the subprocess module to do the following (all within a Jupyter Notebook)

* (A) Create a *Mock_Project* directory within your /2017_Winter_Python/4.2_Modules_Numpy_Scipy folder
* (B) Create *data*, *processed_data*, *scripts*, and *plots* directories.
* (C) Print a list of the files in the *Mock_Project* directory
* (D) Save the list of all files in the Mock_Project directory to a file called 'directory.txt'


__3. BLAST a Fasta File__

Write a function that takes the name of a fasta file as input, and prints the results of a BLAST search of every sequence in the file. Use the __blastn__ option __-outfmt 6__ to give the output in summary tabular form. Test this function with the file *test.fasta* in this directory.

Feel free to modify BLAST code from the above lecture, and to re-use your Fasta parser from this morning's solutions. Bonus: Allow an option to save the output to a file.


__4. Your own BLAST module__

Turn the code you wrote above into your own BLAST module, which should include functions to BLAST a specific sequence, as well as to BLAST all sequences in a given Fasta file.

Save these funcitons to a file, and add the folder you saved your module in to your PYTHONPATH by modifying your **~/.bashrc** file.

Finally, try importing and using your new Python BLAST module.